In [ ]:
import numpy as np
import pandas as pd
from time import time
from netort.data_manager import DataSession
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
%config Application.log_level="INFO"

In [ ]:
# импортируем пхаут
phout_columns = [
    'time', 'tag', 'interval_real',
    'connect_time', 'send_time',
    'latency', 'receive_time',
    'interval_event', 'size_out',
    'size_in', 'net_code', 'proto_code']
data = pd.read_csv(
    "phout.log",
    sep='\t', names=phout_columns)
data[['interval_real',
    'connect_time', 'send_time',
    'latency', 'receive_time',
    'interval_event']]/=1000
data['ts'] = (data["time"] * 1000000 + data["interval_real"]).astype(int)
data['ts'] -= data["ts"][0]
data.size

In [ ]:
def upload_metric(data, name, group=None):
    metric_meta = {
        'type': 'metrics',
        'name': name,
        'hostname': 'localhost',
        'source': 'Jupyter'
    }
    if group:
        metric_meta.update({'group': group})
    metric_obj = data_session.new_metric(metric_meta)
    df = pd.DataFrame()
    df['ts'] = data['ts']
    df['value'] = data[name]
    chunk_count = df.size/1000
    if chunk_count:
        for chunk in np.array_split(df, chunk_count):
            metric_obj.put(chunk)
    else:
        metric_obj.put(df)

In [ ]:
config = {
    'clients': [
        {
            'type': 'luna',
            'api_address': 'https://volta-back.common-int.yandex-team.ru',
            'user_agent': 'Jupyter Test',
        },
        {
            'type': 'local_storage',
        }
    ],
    'test_start': int(time()*1e6),
    'artifacts_base_dir': './logs'
}
data_session = DataSession(config=config)

In [ ]:
upload_metric(data, 'interval_real')

for key in ['connect_time', 'send_time',
    'latency', 'receive_time',
    'interval_event']:
    upload_metric(data, key, group='fractions')
    
for key in ['size_out', 'size_in']:
    upload_metric(data, key, group='size')

In [ ]:
data_session.close()